# Análise do risco de inadimplência dos mutuários

Seu projeto é preparar um relatório para a divisão de empréstimos de um banco. Você precisará descobrir se o estado civil de um cliente e o número de filhos têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

Seu relatório será considerado ao criar uma **pontuação de crédito** de um cliente em potencial. A **contagem de crédito** é usada para avaliar a capacidade de um devedor em potencial de pagar seu empréstimo.

[Neste caderno, você recebe dicas e instruções breves e sugestões de raciocínio. Não os ignore, pois eles são projetados para te equipar com a estrutura do projeto e o ajudarão a analisar o que você está fazendo em um nível mais profundo. Antes de enviar seu projeto, certifique-se de remover todas as dicas e descrições fornecidas a você. Em vez disso, faça com que este relatório pareça que você está enviando para seus colegas de equipe para demonstrar suas descobertas - eles não devem saber que você teve qualquer ajuda externa nossa! Para ajudá-lo, colocamos as dicas que você deve remover entre colchetes.]

[Antes de mergulhar na análise de seus dados, explique os propósitos do projeto e as hipóteses que você testará.]

## Abra o arquivo de dados e veja as informações gerais.

[Comece importando as bibliotecas e carregando os dados. Você pode perceber que precisa de bibliotecas adicionais à medida que avança, o que é totalmente bom - apenas certifique-se de atualizar esta seção quando o fizer.]

In [128]:
# Carregando todas as bibliotecas
import pandas as pd

# Carregue os dados
df = pd.read_csv('/datasets/credit_scoring_eng.csv')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [129]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [130]:
def fill_nan(df, col, parametro_identificado, parametro_identificado_2):
    # Calcula mediana agrupado por 'col'
    df_aux = df.groupby([parametro_identificado, parametro_identificado_2])[col].median().reset_index()
    df_aux[col] = df_aux[col].fillna(df[col].median()) # Preenche vazios, se houver

    # Para cada grupo do agrupamento feito acima
    for i in range(df_aux.shape[0]):
        grupo = df_aux.loc[i, parametro_identificado] # pega o nome do grupo 1
        grupo2 = df_aux.loc[i, parametro_identificado_2] # pega o nome do grupo 2
        valor = df_aux.loc[i, col] # pega o valor da mediana para esse grupo
        
        #Preencha os valores vazios do grupo com o valor da mediana para esse grupo
        df.loc[(df[parametro_identificado]==grupo) & (df[parametro_identificado_2]==grupo2), col] = \
        df.loc[(df[parametro_identificado]==grupo) & (df[parametro_identificado_2]==grupo2), col].fillna(valor)
    
    return df

In [131]:
fill_nan(df, 'total_income', 'income_type', 'gender');

In [132]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [135]:
df.groupby("gender")["total_income"].median()

gender
F      20898.4980
M      26353.5455
XNA    32624.8250
Name: total_income, dtype: float64

In [137]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [141]:
df['total_income'].fillna(df.groupby(["gender", "education", "income_type"])["total_income"].transform("median"))

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21520    35966.698
21521    24959.969
21522    14347.610
21523    39054.888
21524    13127.587
Name: total_income, Length: 21525, dtype: float64

In [136]:
df.groupby(["gender", "education"])["total_income"].transform("median")

0        25366.9865
1        20817.4615
2        25945.7880
3        25945.7880
4        20817.4615
            ...    
21520    20817.4615
21521    20817.4615
21522    25945.7880
21523    25945.7880
21524    20817.4615
Name: total_income, Length: 21525, dtype: float64

In [7]:
df[(df['days_employed'].isna())].shape

(2174, 12)

In [8]:
df[(df['days_employed'].isna()) | (df['total_income'].isna())].shape

(2174, 12)

In [9]:
df[(df[['days_employed', 'total_income']].isna())].shape

(21525, 12)

## Tarefa 1. Exploração de dados

**Descrição dos dados**
- `children` - o número de crianças na família
- `days_employed` - experiência de trabalho em dias
- `dob_years` - idade do cliente em anos
- `education` - educação do cliente
- `education_id` - identificador de educação
- `family_status` - estado civil do cliente
- `family_status_id` - identificador de estado civil
- `gender` - gênero do cliente
- `income_type` - tipo de emprego
- `debt` - havia alguma dívida no pagamento do empréstimo
- `total_income` - renda mensal
- `purpose` - o objetivo de obter um empréstimo

[Agora vamos explorar nossos dados. Você vai querer ver quantas colunas e linhas ele tem, veja algumas linhas para verificar possíveis problemas com os dados.]

In [10]:
# Vamos ver quantas linhas e colunas nosso conjunto de dados tem

df.shape

(21525, 12)

In [11]:
# vamos exibir as primeiras N linhas

df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


[Descreva o que você vê e observa em sua amostra de dados exibida. Há algum problema que possa precisar de mais investigação e alterações?]

In [12]:
# Obter informações sobre dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19448 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


[Existem valores ausentes em todas as colunas ou apenas em algumas? Descreva brevemente o que você vê em 1-2 frases.]

In [13]:
# Vejamos a tabela filtrada com valores ausentes na primeira coluna com dados ausentes
df[df['days_employed'].isna()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


[Os valores ausentes parecem simétricos? Podemos ter certeza desta suposição? Explique brevemente seus pensamentos nesta seção. Você provavelmente pode querer realizar investigações adicionais e contar os valores ausentes em todas as linhas com valores ausentes para confirmar que as amostras ausentes são do mesmo tamanho.]

In [14]:
# Vamos aplicar várias condições para filtrar dados e observar o número de linhas na tabela filtrada.
missing_rows = df[(df['days_employed'].isna()) | (df['total_income'].isna())]
print(missing_rows.shape[0])

2174


**Conclusão intermediária**

[O número de linhas na tabela filtrada corresponde ao número de valores ausentes? Que conclusão podemos tirar disso?]

[Calcule a porcentagem dos valores ausentes em comparação com todo o conjunto de dados. É um pedaço de dados consideravelmente grande? Nesse caso, você pode querer preencher os valores ausentes. Para fazer isso, primeiro devemos considerar se os dados ausentes podem ser devidos à característica específica do cliente, como tipo de emprego ou outra coisa. Você precisará decidir qual característica você acha que pode ser o motivo. Em segundo lugar, devemos verificar se há alguma dependência de valores ausentes em relação ao valor de outros indicadores com as colunas com características identificadas específicas do cliente.]

[Explique seus próximos passos e como eles se correlacionam com as conclusões que você fez até agora.]

In [15]:
# Vamos investigar clientes que não possuem dados sobre as características identificadas e a coluna com os valores ausentes

missing_rows

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [16]:
# Verificar a distribuição

df['days_employed'].value_counts(dropna=False)

 NaN            2174
-1580.622577       1
-4122.460569       1
-2828.237691       1
-2636.090517       1
                ... 
-7120.517564       1
-2146.884040       1
-881.454684        1
-794.666350        1
-3382.113891       1
Name: days_employed, Length: 19352, dtype: int64

In [17]:
df['total_income'].value_counts(dropna=False)

NaN             2077
40804.674606      97
17312.717000       2
31791.384000       2
42413.096000       2
                ... 
26124.613000       1
28692.182000       1
28477.783000       1
26954.608000       1
41428.916000       1
Name: total_income, Length: 19350, dtype: int64

[Descreva suas descobertas aqui.]

**Possíveis motivos para valores ausentes nos dados**

[Proponha suas ideias sobre por que você acha que os valores podem estar ausentes. Você acha que eles estão ausentes aleatoriamente ou há algum padrão?]

[Vamos começar a verificar se os valores ausentes são aleatórios.]

In [18]:
# Verificar a distribuição em todo o conjunto de dados
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2077
purpose                0
dtype: int64

**Conclusão intermediária**

[A distribuição no conjunto de dados original é semelhante à distribuição da tabela filtrada? O que isso significa para nós?]

[Se você acha que ainda não podemos tirar conclusões, vamos investigar melhor nosso conjunto de dados. Vamos pensar em outros motivos que podem levar à ausência de dados e verificar se podemos encontrar algum padrão que possa nos levar a pensar que os valores ausentes não são aleatórios. Como este é o seu trabalho, esta seção é opcional.]

In [19]:
# Verifique outros motivos e padrões que possam levar a valores ausentes



**Conclusão intermediária**

[Podemos finalmente confirmar que os valores ausentes são acidentais? Verifique se há qualquer outra coisa que você acha que pode ser importante aqui.]

In [20]:
# Verificando outros padrões - explique quais

**Conclusões**

[Você encontrou algum padrão? Como você chegou a essa conclusão?]

[Explique como você abordará os valores ausentes. Considere as categorias em que os valores estão ausentes.]

[Planeje brevemente suas próximas etapas para transformar dados. Você provavelmente precisará abordar diferentes tipos de problemas: duplicatas, registros diferentes, artefatos incorretos e valores ausentes.]

## Transformação de dados

[Vamos examinar cada coluna para ver quais problemas podemos ter nelas.]

[Comece removendo duplicatas e corrigindo informações educacionais, se necessário.]

In [21]:
df.drop_duplicates(subset='education')['education']

0         bachelor's degree
1       secondary education
2       Secondary Education
7       SECONDARY EDUCATION
8         BACHELOR'S DEGREE
13             some college
31        primary education
62        Bachelor's Degree
134            SOME COLLEGE
376            Some College
797       PRIMARY EDUCATION
2817      Primary Education
2963        Graduate Degree
4170        GRADUATE DEGREE
6551        graduate degree
Name: education, dtype: object

In [22]:
# Vamos ver todos os valores na coluna de educação para verificar se e quais grafias precisarão ser corrigidas
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [23]:
# Corrija os registros, se necessário


In [24]:
# Verificando todos os valores na coluna para ter certeza de que os corrigimos



[Verifique os dados na coluna `children`]

In [25]:
# Vamos ver a distribuição de valores na coluna `children`
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

[Há alguma coisa estranha na coluna? Se sim, qual é a porcentagem de dados problemáticos? Como elas podem ter ocorrido? Tome uma decisão sobre o que você fará com esses dados e explique seu raciocínio.]

In [26]:
# [corrija os dados com base na sua decisão]


In [27]:
# Verificar a coluna `children` novamente para ter certeza de que está tudo corrigido



[Verifique os dados na coluna `days_employed`. Em primeiro lugar, pense sobre que tipo de problemas pode haver, o que você pode querer verificar e como você fará isso.]

In [28]:
# Encontre dados problemáticos em `days_employed`, se existirem, e calcule a porcentagem


[Se a quantidade de dados problemáticos for alta, pode ser devido a alguns problemas técnicos. Provavelmente, podemos querer propor a razão mais óbvia pela qual isso poderia ter acontecido e quais poderiam ter sido os dados corretos, pois não podemos descartar essas linhas problemáticas.]

In [29]:
# Aborde os valores problemáticos, se existirem



In [30]:
# Verifique o resultado - certifique-se de que está corrigido


[Vamos agora olhar para a idade do cliente e se há algum problema aí. Novamente, pense em quais dados podem ser estranhos nesta coluna, ou seja, quais não podem ser a idade de alguém.]

In [31]:
# Verifique o `dob_years` para valores suspeitos e conte a porcentagem



[Decida o que você fará com os valores problemáticos e explique por quê.]

In [32]:
# Resolva os problemas na coluna `dob_years`, se existirem


In [33]:
# Verifique o resultado - certifique-se de que está corrigido


[Agora vamos verificar a coluna `family_status`. Veja que tipo de valores existem e quais problemas você pode precisar resolver.]

In [34]:
# Vamos ver os valores da coluna



In [35]:
# Aborde os valores problemáticos em `family_status`, se eles existirem



In [36]:
# Verifique o resultado - certifique-se de que está corrigido


[Agora vamos verificar a coluna `gender`. Veja que tipo de valores existem e quais problemas você pode precisar resolver]

In [37]:
# Vamos ver os valores na coluna

In [38]:
# Aborde os valores problemáticos, se existirem

In [39]:
# Verifique o resultado - certifique-se de que está corrigido



[Agora vamos verificar a coluna `income_type`. Veja que tipo de valores existem e quais problemas você pode precisar resolver]

In [40]:
# Vamos ver os valores na coluna

In [41]:
# Aborde os valores problemáticos, se existirem

In [42]:
# Verifique o resultado - certifique-se de que está corrigido



[Agora vamos ver se temos duplicatas em nossos dados. Se o fizermos, você precisará decidir o que fará com eles e explicar o porquê.]

In [43]:
# Verificar duplicatas



In [44]:
# Aborde as duplicatas, se existirem

In [45]:
# Última verificação se temos duplicatas


In [46]:
#Verifique o tamanho do conjunto de dados que você tem agora após suas primeiras manipulações com ele

[Descreva seu novo conjunto de dados: diga brevemente o que mudou e qual é a porcentagem das alterações, se houver.]


# Trabalhando com valores ausentes

[Para acelerar o trabalho com alguns dados, você pode querer trabalhar com dicionários para alguns valores, onde os IDs são fornecidos. Explique por quê e com quais dicionários você trabalhará.]

In [47]:
df['days_employed']

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [48]:
# Encontre os dicionários


### Restaurar valores ausentes em `total_income`

[Informe brevemente quais colunas têm valores ausentes que você precisa resolver. Explique como você irá corrigi-los.]


[Comece abordando os valores ausentes da receita total. Criar e categoria de idade para os clientes. Crie uma nova coluna com a categoria de idade. Essa estratégia pode ajudar no cálculo de valores para a renda total..]


In [49]:
# Vamos escrever uma função que calcule a categoria de idade

df['total_income'].isna().sum()

2077

In [50]:
# Teste se a função funciona


In [51]:
# Criar coluna nova com base na função



In [52]:
# Verificar como os valores na nova coluna



[Pense nos fatores dos quais a renda geralmente depende. Eventualmente, você desejará descobrir se deve usar valores médios ou medianos para substituir valores ausentes. Para tomar essa decisão, você provavelmente vai querer olhar para a distribuição dos fatores que você identificou como tendo impacto na renda de alguém.]

[Crie uma tabela que tenha apenas dados sem valores ausentes. Esses dados serão usados para restaurar os valores ausentes.]

In [53]:
# Crie uma tabela sem valores ausentes e exiba algumas de suas linhas para garantir que ela fique boa

In [54]:
# Veja os valores médios de renda com base em seus fatores identificados

In [55]:
# Veja os valores medianos de renda com base em seus fatores identificados


[Repita essas comparações para vários fatores. Certifique-se de considerar diferentes aspectos e explicar seu processo de pensamento.]



[Tome uma decisão sobre quais características definem mais a renda e se você usará uma mediana ou uma média. Explique por que você tomou essa decisão]


In [56]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [57]:
df_test = df.copy()

In [58]:
df_test.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2077
purpose                0
dtype: int64

In [59]:
#  Escreva uma função que usaremos para preencher os valores ausentes      
df_test.loc[(df_test['gender']=='F') & (df_test['family_status']=='married'), 'total_income'] = df_test.loc[(df_test['gender']=='F') & (df_test['family_status']=='married'), 'total_income'].fillna('median')

In [60]:
# Verifique se funciona
df_test.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        1277
purpose                0
dtype: int64

In [61]:
df_test.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        1277
purpose                0
dtype: int64

In [62]:
df_test['days_employed'].value_counts()

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

In [63]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [64]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [65]:
df.groupby('days_employed')['gender'].count()

days_employed
-18388.949901     1
-17615.563266     1
-16593.472817     1
-16264.699501     1
-16119.687737     1
                 ..
 401663.850046    1
 401674.466633    1
 401675.093434    1
 401715.811749    1
 401755.400475    1
Name: gender, Length: 19351, dtype: int64

In [66]:
df_test.groupby('days_employed')['gender'].count()

days_employed
-18388.949901     1
-17615.563266     1
-16593.472817     1
-16264.699501     1
-16119.687737     1
                 ..
 401663.850046    1
 401674.466633    1
 401675.093434    1
 401715.811749    1
 401755.400475    1
Name: gender, Length: 19351, dtype: int64

In [67]:
df['days_employed'].value_counts()

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

In [68]:
df[df['days_employed']<0].shape[0]

15906

In [69]:
df['total_income'].isna().sum()

2077

In [70]:
df_test = df.copy()

In [71]:
df_test.isna().sum().sum()

4251

In [72]:
def custom_fill_na(df, gender, family_status):
    df.loc[(df['gender']==gender) & (df['family_status']==family_status), 'total_income'] = \
    df.loc[(df['gender']==gender) & (df['family_status']==family_status), 'total_income'].fillna('median')

In [73]:
# Aplique em todas as linhas
custom_fill_na(df_test, 'F', 'married')

In [74]:
df_test.isna().sum().sum()

3451

In [75]:
# Verifique se temos algum erro


[Se você encontrou erros ao preparar os valores para dados ausentes, provavelmente significa que há algo especial nos dados da categoria. Pense um pouco - você pode querer corrigir algumas coisas manualmente, se houver dados suficientes para encontrar medianos/médios.]


In [76]:
# Substituir valores ausentes se houver algum erro


[Quando você achar que terminou com `total_income`, verifique se o número total de valores nesta coluna corresponde ao número de valores em outras.]

In [77]:
# Verificar o número de entradas nas colunas



###  Restaurar valores em `days_employed`

[Pense nos parâmetros que podem ajudá-lo a restaurar os valores ausentes nesta coluna. Eventualmente, você desejará descobrir se deve usar valores médios ou medianos para substituir valores ausentes. Você provavelmente fará uma pesquisa semelhante à que fez ao restaurar dados em uma coluna anterior.]

In [78]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2077
purpose                0
dtype: int64

In [79]:
def fill_nan(df, parametro_identificado):
    # Calcula mediana agrupado por 'days_employed'
    df_aux = df.groupby([parametro_identificado])['days_employed'].median().reset_index()

    # Para cada gruopo do agrupamento feito acima
    for i in range(df_aux.shape[0]):
        grupo = df_aux.loc[i, parametro_identificado] # pega o nome do grupo
        valor = df_aux.loc[i, 'days_employed'] # pega o valor da mediana para esse grupo
        
        #Preencha os valores vazios do grupo com o valor da mediana para esse grupo
        df.loc[df[parametro_identificado]==grupo, 'days_employed'] = df.loc[df[parametro_identificado]==grupo, 'days_employed'].fillna(valor)
    
    return df

In [80]:
fill_nan(df, 'income_type')

df_test.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        1277
purpose                0
dtype: int64

In [81]:
df.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2077
purpose                0
dtype: int64

In [82]:
for k in :
    print(k)

SyntaxError: invalid syntax (737917455.py, line 1)

In [ ]:
# Distribuição de `days_employed` medianos com base em seus parâmetros identificados
df.groupby(['income_type'])['total_income'].median()

In [ ]:
# Distribuição de `days_employed` médios com base em seus parâmetros identificados
df.groupby(['income_type'])['total_income'].mean()

[Decida o que você usará: médias ou medianas. Explique por quê.]

In [ ]:
df['income_type'].value_counts(dropna=False)

In [ ]:
df['income_type']

In [ ]:
# Vamos escrever uma função que calcule médias ou medianas (dependendo da sua decisão) com base no seu parâmetro identificado
def calcular_media_por_categoria(df, coluna='income_type')

In [ ]:
df['days_employed'].mean()

In [ ]:
df['days_employed'] = df['days_employed'].abs()

In [ ]:
df_aux = df[['income_type', 'days_employed']].groupby('income_type').median()
df_aux.loc['business'].iloc[0]

In [ ]:
df['days_employed'].fillna('')

In [ ]:
df.apply(lambda x: df_aux.loc[x['income_type']].iloc[0])

In [ ]:
df['days_employed'].isna().sum()

In [ ]:
# Verifique se a função funciona



In [ ]:
# Aplicar função ao income_type



In [ ]:
# Verifique se a função funcionou



In [ ]:
# Substituir valores ausentes



[Quando você achar que terminou com `total_income`, verifique se o número total de valores nesta coluna corresponde ao número de valores em outras.]

In [ ]:
# Verifique as entradas em todas as colunas - certifique-se de corrigir todos os valores ausentes

## Categorização de dados

[Para responder às perguntas e testar as hipóteses, você vai querer trabalhar com dados categorizados. Veja as perguntas que lhe foram colocadas e que você deve responder. Pense em quais dados precisarão ser categorizados para responder a essas perguntas. Abaixo, você encontrará um modelo por meio do qual poderá trabalhar à sua maneira ao categorizar os dados. O primeiro processamento passo a passo abrange os dados de texto; o segundo aborda os dados numéricos que precisam ser categorizados. Você pode usar ambas ou nenhuma das instruções sugeridas - você decide.]

[Independente de como você decidir abordar a categorização, certifique-se de fornecer uma explicação clara do por quê você tomou sua decisão. Lembre-se: este é o seu trabalho e você toma todas as decisões nele.]


In [ ]:
## Exiba os valores dos dados selecionados para categorização

df

[Vamos verificar valores exclusivos]

In [ ]:
# Verifique os valores exclusivos

In [ ]:
df

[Quais grupos principais você pode identificar com base nos valores exclusivos?]

[Com base nesses temas, provavelmente desejaremos categorizar nossos dados.]


In [ ]:
df.apply()

In [ ]:
# Vamos escrever uma função para categorizar os dados com base em tópicos comuns


df['']
df.loc[(df['children']<=3) & (df['family_status']=='civil partnership')]

In [ ]:
# Crie uma coluna com as categorias e conte os valores para elas



[Se você decidir categorizar os dados numéricos, também precisará criar as categorias para isso.]

In [ ]:
# Examinar todos os dados numéricos em sua coluna selecionada para categorização


In [ ]:
# Obter estatísticas resumidas para a coluna



[Decida quais intervalos você usará para agrupar e explique por quê.]

In [ ]:
# Criar função para categorização em diferentes grupos numéricos com base em intervalos



In [ ]:
# Criar coluna com categorias


In [ ]:
# Conte os valores de cada categoria para ver a distribuição


## Verificar as Hipóteses


**Existe uma correlação entre o nível de renda e do pagamento em dia?**

In [ ]:
# Verifique os dados das crianças e do pagamento em dia


# Calcular a taxa de inadimplência com base no número de filhos



**Conclusão**

[Escreva suas conclusões com base em suas manipulações e observações.]


**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [ ]:
# Verifique os dados de status da família e do pagamento em dia



# Calcular a taxa padrão com base no status da família



**Conclusão**

[Escreva suas conclusões com base em suas manipulações e observações.]

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [ ]:
# Verifique os dados do nível de renda e do pagamento em dia



# Calcular a taxa de inadimplência com base no nível de renda



**Conclusão**

[Escreva suas conclusões com base em suas manipulações e observações.]

**Como a finalidade do crédito afeta a taxa de inadimplência?**

In [ ]:
# Confira os percentuais de inadimplência para cada finalidade de crédito e analise-os



**Conclusão**

[Escreva suas conclusões com base em suas manipulações e observações.]


# Conclusão Geral 

[Liste suas conclusões nesta seção final. Certifique-se de incluir todas as suas conclusões importantes que o levaram à maneira como você processou e analisou os dados. Cubra os valores ausentes, duplicatas e possíveis razões e soluções para artefatos problemáticos que você teve que resolver.]

[Liste suas conclusões sobre as perguntas feitas aqui também.]
